In [1]:
import pandas as pd

# Load the data
file_path = '../data/MachineLearningRating_v3.txt'  
data = pd.read_csv(file_path, delimiter='|')  

print(data.head())

C:\Users\HP\AppData\Local\Temp\ipykernel_17692\3744290036.py:5: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter='|')


   UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
0               145249     12827  2015-03-01 00:00:00             True   
1               145249     12827  2015-05-01 00:00:00             True   
2               145249     12827  2015-07-01 00:00:00             True   
3               145255     12827  2015-05-01 00:00:00             True   
4               145255     12827  2015-07-01 00:00:00             True   

  Citizenship          LegalType Title Language                 Bank  \
0              Close Corporation    Mr  English  First National Bank   
1              Close Corporation    Mr  English  First National Bank   
2              Close Corporation    Mr  English  First National Bank   
3              Close Corporation    Mr  English  First National Bank   
4              Close Corporation    Mr  English  First National Bank   

       AccountType  ...                    ExcessSelected CoverCategory  \
0  Current account  ...             Mobility - 

In [2]:
print(data.columns.tolist())

['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']


In [ ]:
# Get the two most common provinces
top_provinces = data['Province'].value_counts().index[:2]

group_prov_1 = data[data['Province'] == top_provinces[0]]
group_prov_2 = data[data['Province'] == top_provinces[1]]

: 

In [ ]:
# Get top 2 zip codes with the most records
top_postal_codes = data['PostalCode'].value_counts().index[:2]

group_zip_1 = data[data['PostalCode'] == top_postal_codes[0]]
group_zip_2 = data[data['PostalCode'] == top_postal_codes[1]]

: 

In [ ]:
group_female = data[data['Gender'].str.lower() == 'female']
group_male = data[data['Gender'].str.lower() == 'male']

: 

In [ ]:
from scipy.stats import chi2_contingency

# Step 1: Create the binary HasClaim column
data['HasClaim'] = data['TotalClaims'] > 0

# Step 2: Get the top 2 provinces
top_provinces = data['Province'].value_counts().index[:2]
prov1 = top_provinces[0]
prov2 = top_provinces[1]

# Step 3: Subset data
group_prov1 = data[data['Province'] == prov1]
group_prov2 = data[data['Province'] == prov2]

# Step 4: Build a contingency table
contingency = [
    [
        group_prov1['HasClaim'].sum(),                      # number of claims in prov1
        len(group_prov1) - group_prov1['HasClaim'].sum()    # no-claims in prov1
    ],
    [
        group_prov2['HasClaim'].sum(),                      
        len(group_prov2) - group_prov2['HasClaim'].sum()
    ]
]

# Step 5: Perform chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency)

print(f"Chi-squared: {chi2:.2f}")
print(f"p-value: {p:.4f}")

: 

In [7]:
from scipy.stats import ttest_ind

# Step 1: Filter for policies that had claims
claims_data = data[data['TotalClaims'] > 0]

# Step 2: Get claim values by province
claims_prov1 = claims_data[claims_data['Province'] == prov1]['TotalClaims']
claims_prov2 = claims_data[claims_data['Province'] == prov2]['TotalClaims']

# Step 3: Perform independent t-test
t_stat, p_value = ttest_ind(claims_prov1, claims_prov2, equal_var=False)  # Welch's t-test

print(f"T-statistic: {t_stat:.2f}")
print(f"p-value: {p_value:.4f}")

T-statistic: -2.17
p-value: 0.0306


In [8]:
# Step 1: Get top 2 postal codes
top_postal_codes = data['PostalCode'].value_counts().index[:2]
zip1 = top_postal_codes[0]
zip2 = top_postal_codes[1]

# Step 2: Subset data
group_zip1 = data[data['PostalCode'] == zip1]
group_zip2 = data[data['PostalCode'] == zip2]

# Step 3: Build contingency table for claim frequency
contingency_zip = [
    [
        group_zip1['HasClaim'].sum(),
        len(group_zip1) - group_zip1['HasClaim'].sum()
    ],
    [
        group_zip2['HasClaim'].sum(),
        len(group_zip2) - group_zip2['HasClaim'].sum()
    ]
]

# Step 4: Perform chi-squared test
chi2_zip, p_zip, dof_zip, expected_zip = chi2_contingency(contingency_zip)

print(f"Chi-squared: {chi2_zip:.2f}")
print(f"p-value: {p_zip:.4f}")

Chi-squared: 3.60
p-value: 0.0579


In [9]:
# Step 1: Filter to policies with claims
claims_data_zip = data[data['TotalClaims'] > 0]

# Step 2: Subset data by top 2 postal codes
claims_zip1 = claims_data_zip[claims_data_zip['PostalCode'] == zip1]['TotalClaims']
claims_zip2 = claims_data_zip[claims_data_zip['PostalCode'] == zip2]['TotalClaims']

# Step 3: Perform Welch's t-test (assumes unequal variance)
t_stat_zip, p_val_zip = ttest_ind(claims_zip1, claims_zip2, equal_var=False)

print(f"T-statistic: {t_stat_zip:.2f}")
print(f"p-value: {p_val_zip:.4f}")

T-statistic: 0.39
p-value: 0.7002


In [10]:
# Step 1: Create margin column
data['Margin'] = data['TotalPremium'] - data['TotalClaims']

# Step 2: Subset margin by top 2 zip codes
margin_zip1 = data[data['PostalCode'] == zip1]['Margin']
margin_zip2 = data[data['PostalCode'] == zip2]['Margin']

# Step 3: Perform Welch's t-test
t_stat_margin, p_val_margin = ttest_ind(margin_zip1, margin_zip2, equal_var=False)

print(f"T-statistic: {t_stat_margin:.2f}")
print(f"p-value: {p_val_margin:.4f}")

T-statistic: 1.16
p-value: 0.2445


In [11]:
# Step 1: Filter out rows with missing gender
gender_data = data[data['Gender'].isin(['Male', 'Female'])]

# Step 2: Build contingency table
contingency_gender = pd.crosstab(gender_data['Gender'], gender_data['HasClaim'])

# Step 3: Perform chi-squared test
chi2_gender, p_gender, _, _ = chi2_contingency(contingency_gender)

print(f"Chi-squared: {chi2_gender:.2f}")
print(f"p-value: {p_gender:.4f}")

Chi-squared: 0.00
p-value: 0.9515


In [12]:
# Step 1: Filter policies with claims and valid gender
claims_gender = data[(data['TotalClaims'] > 0) & (data['Gender'].isin(['Male', 'Female']))]

# Step 2: Separate claim amounts by gender
claims_male = claims_gender[claims_gender['Gender'] == 'Male']['TotalClaims']
claims_female = claims_gender[claims_gender['Gender'] == 'Female']['TotalClaims']

# Step 3: Perform Welch's t-test (unequal variance)
t_stat_gender, p_val_gender = ttest_ind(claims_male, claims_female, equal_var=False)

print(f"T-statistic: {t_stat_gender:.2f}")
print(f"p-value: {p_val_gender:.4f}")

T-statistic: -0.58
p-value: 0.5680
